In [20]:
from sqlalchemy import create_engine
from pydantic import BaseSettings, Field
import pandas as pd
import dotenv
from sqlalchemy.sql import text

dotenv.load_dotenv()

True

In [21]:
class Config(BaseSettings):
    user: str = Field(env='user')
    pasw: str = Field(env='pasw')
    host: str = Field(env='host')
    port: int = Field(env='port')
    db: str = Field(env='db')
    
    @property 
    def ddl(self):
        return f'postgresql+psycopg2://{self.user}:{self.pasw}@{self.host}:{self.port}/{self.db}'
    
config = Config()

engine = create_engine(config.ddl)

In [16]:
query = '''
WITH profile AS
  (SELECT u.user_id,
          DATE_TRUNC('month', MIN(event_time))::date AS start_mn
   FROM tools_shop.users u
   JOIN tools_shop.orders o ON u.user_id = o.user_id
   JOIN tools_shop.events e ON u.user_id = e.user_id
   GROUP BY 1), 
sessions AS
  (SELECT start_mn,
          DATE_TRUNC('month', event_time)::date AS event_mn,
          COUNT(DISTINCT p.user_id) AS users_cnt
   FROM profile p
   JOIN tools_shop.events e ON p.user_id = e.user_id
   GROUP BY 1,
            2)
SELECT start_mn,
       event_mn,
       users_cnt,
       LAG(users_cnt) OVER (PARTITION BY start_mn ORDER BY event_mn) AS previous_day_users_cnt,
       ROUND((1 - (users_cnt::numeric / LAG(users_cnt) OVER (PARTITION BY start_mn ORDER BY event_mn))) * 100, 2) AS churn_rate
FROM sessions
WHERE start_mn BETWEEN '2018-01-01' AND '2018-06-01'
  AND event_mn BETWEEN '2018-01-01' AND '2018-12-01'; 
'''

In [18]:
with engine.connect() as conn:
    df = pd.read_sql(text(query), con=conn)

In [23]:
df

,start_mn,event_mn,users_cnt,previous_day_users_cnt,churn_rate
0,2018-01-01,2018-01-01,257,NaN,NaN
1,2018-01-01,2018-02-01,67,257.0,73.93
2,2018-01-01,2018-03-01,41,67.0,38.81
3,2018-01-01,2018-04-01,44,41.0,-7.32
4,2018-01-01,2018-05-01,42,44.0,4.55
5,2018-01-01,2018-06-01,21,42.0,50.00
6,2018-01-01,2018-07-01,11,21.0,47.62
7,2018-01-01,2018-08-01,3,11.0,72.73
8,2018-02-01,2018-02-01,226,NaN,NaN
9,2018-02-01,2018-03-01,59,226.0,73.89
